## Filtering in Hadamard Domain

In [1]:
from scipy.linalg import hadamard
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [2]:
# Functions that we need

In [3]:
int2bin = lambda x, n: format(x, 'b').zfill(n)

In [4]:
#
# The function generates an ideal low pass filter for frequency domain, 
# M and N are size of the filter/image, D0 is the cut_off point. 
# 
def idealLowPass(M, N, D0):
    # Initializing the filter with ones; since the filter is a complex function,
    # it has two channels, representing the real and imaginary parts:
    filter = np.ones((M, N), dtype=np.uint8)
    D0 = min(M,N) / 2 * D0
    # Scanning through each pixel and calculating the distance of each pixel
    # to the image center. If the pixel is within D0, it is changed to 0:
    for i in range(M):
        for j in range(N):
            if ( (i-M/2)**2 + (j-N/2)**2)**0.5 >= D0:
                filter[i,j]= 0
            
    return filter


In [5]:
def horder(b,nn): 
    jj = int2bin(b,nn)
    kk = ''
    for j in range(nn): 
        kk = kk+jj[nn-1-j] 
    
    kkk=np.zeros(nn) 
    kkk[0] = kk[0] 
    for j in range(1,nn):
        kkk[j] = int(kkk[j-1]) ^ int(kk[j]) 
        
    k=0
    for j in range(nn):
        k = k + int(kkk[j]) * 2**(nn-1-j)  

    return int(k)

In [6]:
# h = ordhad(n) 
# generate a n*n ordered hadamard matrix
# amir - may 2022
#

def ordhad(n): 
    h = hadamard(n)
    hh = hadamard(n)
    nn = np.log2(n)
    for i in range(n):
        k = horder(int(i) , int(nn)) 
        hh[k][:] = h[i][:]

    return hh

In [7]:
# 
# to generate a lowpass Butterworth filter of the frequency domain  
# filter size is MxN, cut_off point of the filter is D0, n_o is the filter order
#
def ButterworthLowPass(M, N, D0, n_o):
    #  
    filter = np.zeros((M, N))
    # normalized cut_off frequency is mapped to real index
    D0 = D0 * min(M,N) / 2
    n_o = 2 * n_o
    for i in range(M):
        for j in range(N):
            d = ( (i-M/2)**2 + (j-N/2)**2 )**0.5
            filter[i,j]= 1 / ( 1 + (d/D0)**n_o )
            
    return filter


In [8]:
def ready_2_show(a, level=255):
    a = ( a - np.min(a) ) / (np.max(a) - np.min(a)) 
    a = a * level
    return np.uint8(a)


In [9]:
# setting the cut_off frequency to ...
ctf= 0.2      # lower cut-off frequency
ctf2 = 0.4    # higher cut-off frequency
MM = 1024     # we convert the input image into MMxMM size

In [10]:
# ordered Hadamard Matrix
h = ordhad(MM)


In [11]:
# reading and pre-processing of the image, it will be graylevel and will be converted to MMxMM pixel
img = cv2.imread('NINTCHDBPICT0000009753231.jpg')

In [12]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(MM,MM))

In [13]:
# Hadamard Transform
HI = np.matmul( h, np.matmul(img,h) )

In [14]:
'''There are three main ways to perform NumPy matrix multiplication:
dot(array a, array b) : returns the scalar or dot product of two arrays.
matmul(array a, array b) : returns the matrix product of two arrays.
multiply(array a, array b) : returns the element-wise matrix multiplication of two arrays.'''

'There are three main ways to perform NumPy matrix multiplication:\ndot(array a, array b) : returns the scalar or dot product of two arrays.\nmatmul(array a, array b) : returns the matrix product of two arrays.\nmultiply(array a, array b) : returns the element-wise matrix multiplication of two arrays.'

In [15]:
# our filters are designed for the Fourier domain, wher ethe origin of frequency access is
# in the middle of the matrix. in Hadamard and Cosine cases, that origin is in the [0,0] corner of the matrix
# so we go for a double filter size (here 1024x1024), but only keep its 4th quarter [512:1024,512:1024]
# that quarter is a good filter for Hadamard domain. 
# first, lets try a lowpass butterworth filter in hadamard domain, cut-off is ctf, order=2
#
h_lp_filt = ButterworthLowPass(MM*2,MM*2,ctf,2)
h_lp_filt = h_lp_filt[MM:2*MM,MM:2*MM]
print(h_lp_filt.shape) 


(1024, 1024)


In [16]:
# filtering ...
HI = np.multiply(HI,h_lp_filt)

In [17]:
# inverse Hadamard, constant coeeficient is 1/ image size, here: MM**2
#
h2 = np.dot( np.matmul( h, np.matmul(HI,h)) , 1/(MM**2) )
print(h2)

[[137.83399228 137.76494762 137.94250867 ...  95.44127005  97.5342106
   97.56145154]
 [137.80959972 137.72534318 137.87479065 ...  95.395283    97.50537181
   97.52593467]
 [137.92767287 137.83359044 137.70856078 ...  94.6228716   96.84434745
   96.9226112 ]
 ...
 [117.55871201 117.53361321 116.01238734 ... 150.0949558  149.41349351
  149.38364227]
 [117.53615236 117.51818063 116.19365283 ... 149.4933185  148.86594543
  148.82055764]
 [117.53269661 117.51107918 116.15347362 ... 149.51500411 148.86534142
  148.82264238]]


In [18]:

# showing the result
cv2.namedWindow("Hadamard BW-Cyl LP Filtered", cv2.WINDOW_NORMAL)
cv2.imshow("Hadamard BW-Cyl LP Filtered",np.uint8(h2))
cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.imshow("image",img)


In [19]:
# Hadamard Bandpass filtering, Ideal
# Had Transform
HI = np.matmul( h, np.matmul(img,h) )
# building the filter
# a double size LP filter, ideal, cut-off=ctf
filt1 = idealLowPass(MM*2,MM*2,ctf)
# keeping quarter4 of that
filt1 = filt1[MM:2*MM,MM:2*MM]
# a double size LP filter, ideal, cut-off=ctf2, where ctf2>ctf
filt2 = idealLowPass(MM*2,MM*2,ctf2)
# keeping quarter4 of that
filt2 = filt2[MM:2*MM,MM:2*MM]
# configuration of a BP filter for Hadamard domain
filt2 = filt2 - filt1
# filtering ...
HI = np.multiply(HI,filt2)
# inverse Hadamard, constant coeeficient is 1/ image size, result in h2
h2 = np.dot( np.matmul( h, np.matmul(HI,h)) , 1/(MM**2) )


In [20]:
# Hadamard Bandreject filtering, Ideal
# Had Transform
HI = np.matmul( h, np.matmul(img,h) )
# building the band-reject filter, which is indeed 1- bandpass filter
filt2 = 1 - filt2 
# filtering ...
HI = np.multiply(HI,filt2)
# inverse Hadamard, constant coeeficient is 1/ image size, result in h3
h3 = np.dot( np.matmul( h, np.matmul(HI,h)) , 1/(MM**2) )


In [21]:
# showing the result
# bandpass filtering result
cv2.namedWindow("Hadamard ideal BP Filt", cv2.WINDOW_NORMAL)
cv2.imshow("Hadamard ideal BP Filt",np.uint8(h2))

#bandreject filtering result
cv2.namedWindow("Hadamard ideal BR Filt", cv2.WINDOW_NORMAL)
cv2.imshow("Hadamard ideal BR Filt",np.uint8(h3) )

# the filter
cv2.namedWindow("The Filter", cv2.WINDOW_NORMAL)
cv2.imshow("The Filter",np.uint8(filt2*200))

# ready_to_show function normalizes the elements between zero and level, default: level=255
# showing the normalized bandpass filtering result
cv2.namedWindow("Hadamard ideal BP Filt, ready to show", cv2.WINDOW_NORMAL)
cv2.imshow("Hadamard ideal BP Filt, ready to show",ready_2_show(h2))

In [22]:

cv2.waitKey(0)
cv2.destroyAllWindows() 

In [23]:
# differences between the image and its band-rejected version
img - h3 

array([[ -4.82155323,  -4.82155323, -10.71194744, ...,   2.08487225,
         21.360116  ,  21.360116  ],
       [ -4.82155323,  -4.82155323, -10.71194744, ...,   2.08487225,
         21.360116  ,  21.360116  ],
       [-15.77704906, -15.77704906, -22.82395267, ...,  -6.60941601,
         13.90565014,  13.90565014],
       ...,
       [  6.63051891,   6.63051891,  -4.02652264, ...,   4.68452358,
         -0.40146923,  -0.40146923],
       [  6.54261875,   6.54261875,  -4.2887392 , ...,   2.40954304,
         -2.18800068,  -2.18800068],
       [  6.54261875,   6.54261875,  -4.2887392 , ...,   2.40954304,
         -2.18800068,  -2.18800068]])